# TODO
https://iver56.github.io/audiomentations/waveform_transforms/add_short_noises/


https://iver56.github.io/audiomentations/waveform_transforms/aliasing/


https://iver56.github.io/audiomentations/waveform_transforms/apply_impulse_response/ + jakiś dataset


https://iver56.github.io/audiomentations/waveform_transforms/gain_transition/


https://iver56.github.io/audiomentations/waveform_transforms/high_pass_filter/


https://iver56.github.io/audiomentations/waveform_transforms/high_shelf_filter/


https://iver56.github.io/audiomentations/waveform_transforms/limiter/

Kompresje
https://iver56.github.io/audiomentations/waveform_transforms/mp3_compression/ i inne

augumentacje on-line: Mixup, SpecAugment (po prostu)

Filtry skonczonej odpowiedzi impulsowej


RawBoost https://arxiv.org/abs/2111.04433


Dodanie paplaniny

In [ ]:
import librosa
import numpy as np
import soundfile as sf
from audiomentations import Compose, BandStopFilter, TimeMask, AddGaussianNoise, TanhDistortion, BandPassFilter, BitCrush, \
                            Gain, PitchShift, TimeStretch
from scipy.signal import fftconvolve

In [ ]:
def load_audio(audio):
    y, sr = librosa.load(audio, sr=None)
    return y, sr

def apply_frequency_masking(audio, sample_rate):
    augmenter = Compose([
        BandStopFilter(
            min_center_freq=200.0,
            max_center_freq=4000.0,
            min_bandwidth_fraction=0.5,
            max_bandwidth_fraction=1.99,
            p=1.0
        )
    ])
    augmented = augmenter(samples=audio, sample_rate=sample_rate)
    return augmented

def apply_time_masking(audio, sample_rate):
    augmenter = Compose([
        TimeMask(min_band_part=0.05, max_band_part=0.2, p=1.0)
    ])
    augmented = augmenter(samples=audio, sample_rate=sample_rate)
    return augmented

def apply_mixup(audio1, audio2, proportion=0.5):
    min_len = min(len(audio1), len(audio2))
    audio1, audio2 = audio1[:min_len], audio2[:min_len]

    mixed_audio = proportion * audio1 + (1 - proportion) * audio2
    return mixed_audio

def apply_volume_increase(audio, sample_rate, min_gain_in_db=6.0, max_gain_in_db=6.0):
    augmenter = Compose([
        Gain(min_gain_in_db=min_gain_in_db, max_gain_in_db=max_gain_in_db, p=1.0)
    ])
    augmented = augmenter(samples=audio, sample_rate=sample_rate)
    return augmented

def apply_speed_increase(audio, sample_rate, min_rate=1.1, max_rate=1.3):
    augmenter = Compose([
        TimeStretch(min_rate=min_rate, max_rate=max_rate, p=1.0)
    ])
    augmented = augmenter(samples=audio, sample_rate=sample_rate)
    return augmented

def apply_pitch_shift(audio, sample_rate, min_semitones=-4, max_semitones=4):
    augmenter = Compose([
        PitchShift(min_semitones=min_semitones, max_semitones=max_semitones, p=1.0)
    ])
    augmented = augmenter(samples=audio, sample_rate=sample_rate)
    return augmented

def apply_white_noise(audio, sample_rate, min_amplitude=0.001, max_amplitude=0.015):
    augmenter = Compose([
        AddGaussianNoise(min_amplitude=min_amplitude, max_amplitude=max_amplitude, p=1.0),
    ])
    augmented_audio = augmenter(samples=audio, sample_rate=sample_rate)
    return augmented_audio

def apply_tanh_distortion(audio, sample_rate, min_distortion=0.01, max_distortion=0.7, p=1.0):
    augmenter = Compose([
        TanhDistortion(
            min_distortion=min_distortion,
            max_distortion=max_distortion,
            p=p
        )
    ])
    augmented_sound = augmenter(samples=audio, sample_rate=sample_rate)
    return augmented_sound

def apply_band_pass_filter(audio, sample_rate, min_center_freq=100.0, max_center_freq=6000.0, p=1.0):
    augmenter = Compose([
        BandPassFilter(min_center_freq=min_center_freq, max_center_freq=max_center_freq, p=p)
    ])
    augmented_sound = augmenter(samples=audio, sample_rate=sample_rate)
    return augmented_sound

def apply_bit_crush(audio, sample_rate, min_bit_depth=5, max_bit_depth=14, p=1.0):
    augmenter = Compose([
        BitCrush(min_bit_depth=min_bit_depth, max_bit_depth=max_bit_depth, p=p)
    ])
    augmented_sound = augmenter(samples=audio, sample_rate=sample_rate)
    return augmented_sound

def apply_vibrato(audio, sample_rate, vibrato_freq=5, vibrato_magnitude=0.003):
    t = np.arange(len(audio)) / sample_rate
    modulation = vibrato_magnitude * np.sin(2 * np.pi * vibrato_freq * t)
    indices = np.arange(len(audio)) + modulation * sample_rate
    
    indices = np.clip(indices, 0, len(audio) - 1)
    vibrato_audio = np.interp(indices, np.arange(len(audio)), audio)
    
    return vibrato_audio

def apply_reverb(audio, sample_rate, reverb_decay=0.5, reverb_delay_ms=50):
    delay_samples = int(sample_rate * reverb_delay_ms / 1000)
    impulse_response = np.zeros(delay_samples + 1)
    impulse_response[0] = 1.0
    impulse_response[-1] = reverb_decay
    
    reverbed = fftconvolve(audio, impulse_response, mode='full')[:len(audio)]
    
    reverbed = reverbed / np.max(np.abs(reverbed))
    return reverbed

def save_to_audio(wave, sample_rate, filename):
    sf.write(filename, wave, sr=sample_rate)

In [ ]:
file1 = "./data/examples/real/GOLEC UORKIESTRA - ŚCIERNISCO (Official Video ⧸ Remastered).mp3"
file2 = "./data/examples/suno/Warsaw Dream.mp3"

y1, sr = load_audio(file1)
y2, sr = load_audio(file2)

In [ ]:
masked_freq = apply_frequency_masking(y1, sr)
masked_time = apply_time_masking(y1, sr)

sf.write("./masked/freq_masked.wav", masked_freq, sr)
sf.write("./masked/time_masked.wav", masked_time, sr)

In [ ]:
mixed = apply_mixup(y1, y2)
sf.write("./masked/mixup.wav", mixed, sr)

In [ ]:
audio_louder = apply_volume_increase(y1, gain_db=-6)
audio_faster = apply_speed_increase(y1, sr, speed_factor=1.5)
audio_pitch_up = apply_pitch_shift(y1, sr, n_steps=4)

sf.write("./masked/audio_louder.wav", audio_louder, sr)
sf.write("./masked/audio_faster.wav", audio_faster, sr)
sf.write("./masked/audio_pitch_up.wav", audio_pitch_up, sr)

In [ ]:
audio_noisy = apply_white_noise(y1, sr)
sf.write("./masked/audio_with_white_noise.wav", audio_noisy, sr)

In [ ]:
audio_vibrato = apply_vibrato(y1, sr, reverb_decay=0.75)
sf.write("./masked/vibrato.wav", audio_vibrato, sr)